In [11]:
!pip install pandas numpy matplotlib seaborn scikit-learn kaggle transformers torch datasets accelerate -U

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch


In [2]:
from datasets import load_dataset

dataset = load_dataset("GonzaloA/fake_news")


C:\Users\Chen\anaconda3\envs\FinalCyberAttacksDetectionEx\Lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [3]:
from transformers import LongformerTokenizer

tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')


# Function to tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=1024)


# Apply tokenization to all splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [4]:
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [7]:
from transformers import LongformerForSequenceClassification, Trainer, TrainingArguments

model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
results = trainer.evaluate(tokenized_datasets['test'])
print(results)
